# Introduction and basic usage of the NCrystal Python API

This notebook describes the basic typical usage of NCrystal Python API, and lays the foundation for further more detailed tutorials.

## Preamble
Install dependencies and prepare plots. Feel free to edit as you wish:

In [ ]:
#By default we only do pip installs on Google Colab, but you
#can set the variable in the next line to True if you need it:
always_do_pip_installs = False
try:
    import google.colab as google_colab
except ModuleNotFoundError:
    google_colab=None#not on google colab
if always_do_pip_installs or google_colab:
    from importlib.util import find_spec as _fs
    if not _fs('NCrystal'):
        %pip -q install ncrystal ipympl numpy matplotlib
#enable inline and interactive matplotlib plots:
if google_colab:
    google_colab.output.enable_custom_widget_manager()
%matplotlib ipympl
import matplotlib
matplotlib.rcParams.update({"figure.autolayout": True})
#always import NCrystal:
import NCrystal as NC
assert NC.version_num >=  3009006, "too old NCrystal found"
NC.test() #< quick unit test that installation works

## Using basic NCrystal objects
Jumping right into it, let us load a material defined by a *cfg-string* and plot the cross sections. Most often, a cfg-string is a data file name, and perhaps some parameters - like for setting the temperature:

In [ ]:
mat = NC.load('Al_sg225.ncmat;temp=10C')

In reality, the `NC.load` call in the previous line has actually combined into `mat` three kinds of NCrystal objects, that together provides all the information and capabilities that NCrystal provides for that material: An `Info` object, a `Scatter` object and an `Absorption` object. The `Info` object provides general information about the material like density and composition, as well as details about the static structure and dynamics of the material. The `Scatter` and `Absorption` objects provides algorithms concerning neutron interactions in the material. The `mat` object itself contains a few convenience utilities. For instance, we can inspect neutron cross section values:

In [ ]:
mat.plot()

The entire Python API of NCrystal is enriched with doc-strings for inline help, let us learn more about this `mat` material object:

In [ ]:
help(mat)

Indeed, we can see that there are essentially `.info`, `.absorption`, and `.scatter` objects as properties as well as convenience methods `.plot()`, `.dump()`, `.xsect(..)` and `.macroscopic_xsect(..)`. The two latter sums up cross sections of the `.absorption` and `.scatter` objects. We check that we can access the more fundamental objects:

In [ ]:
print( mat.info )
print( mat.scatter )
print( mat.absorption )

And here we request a brief printout of the three objects:

In [ ]:
mat.dump()

The `NC.load(..)` method is merely a convenience method for interactive work. If you only need a particular object you can instead call `createInfo`. `createScatter` or `createAbsorption` directly:

In [ ]:
info_Al = NC.createInfo('Al_sg225.ncmat')
info_Al.dump()

If you are interested in getting a bit more info, you can increase the verbosity (up to 2). This will affect the HKL listing and (for very complex materials) the list of atom positions:

In [ ]:
info_Al.dump(verbose=1)

For more rich plotting of cross sections, the `plot_xsects` and `plot_xsect` functions from the `NCrystal.plot` module can be used. For instance, dive into the different scattering cross section components:

In [ ]:
import NCrystal.plot as ncplot
ncplot.plot_xsect('Al_sg225.ncmat')#supply the argument show_absorption=False to hide absorption

Or perhaps you prefer macroscopic cross sections (a.k.a. inverse penetration depth). It is of course just scaled with the number density (atoms/volume) so only the Y-axis scale and text changes:

In [ ]:
ncplot.plot_xsect('Al_sg225.ncmat',xsmode='macroscopic')

If you wish to overlay different materials and compare cross sections, you have to use the `plot_xsects` (plural `s` at the end) function instead:

In [ ]:
ncplot.plot_xsects('Al_sg225.ncmat;temp=100K',
                   'Al_sg225.ncmat;temp=300K',
                   'Al_sg225.ncmat;temp=500K')

## Simple usage of Scatter or Absorption objects

If you need to extract cross section values, you can do so with the `Scatter` and `Absorption` objects. Note that NCrystal use electronvolt (eV) and angstrom (Å) as default units for energy and lengths respectively:

In [ ]:
scatter_Al = NC.createScatter('Al_sg225.ncmat')#NB: default temperature for most materials is 293.15K
print( '%g barn'%scatter_Al.xsect(wl=1.8) )#query at a single neutron wavelength
print( '%g eV'%NC.wl2ekin(1.8) )#NB: ekin2wl exists as well, plus many more conversion fcts in NCrystal.constants module
print( '%g barn'%scatter_Al.xsect(ekin=0.025248212841151512) )#query at a single neutron energy

We can use numpy arrays as well:

In [ ]:
import numpy
wavelengths = numpy.linspace(0.0,10.0,20)
print( wavelengths )
print( scatter_Al.xsect(wl=wavelengths) )

To extract single components, we must create a scatter instance with just that component, using the cfg-string parameter `comp` (+changing the temperature to show the usage of multiple parameters):

In [ ]:
scatter_Al_only_bragg = NC.createScatter('Al_sg225.ncmat;comp=bragg;temp=100K')
print( '%g barn'%scatter_Al_only_bragg.xsect(wl=4.0) )
scatter_Al_only_bragg.plot()

In addition to extracting cross sections with both `Scatter` and `Absorption` objects, one can of course also use a `Scatter` object to perform random sampling of scattering events by using the `.scatter` method. As usual, we can use the builtin `help` function to figure out how:

In [ ]:
help(scatter_Al)
help(scatter_Al.scatter)
help(scatter_Al.sampleScatterIsotropic)

And we can do a little plot of scattering events for a 1.8Å neutron (assuming a very thin material with no multiple scattering):

In [ ]:
ekin_i = NC.wl2ekin(1.8)
ekin_f, mu = scatter_Al.scatter(ekin=ekin_i,repeat=int(1e6))
import matplotlib
import matplotlib.pyplot as plt
plt.figure()
plt.hist2d((ekin_f-ekin_i)*1000,mu,
           bins=80,
           norm=matplotlib.colors.LogNorm())
plt.ylim(-1,1)
plt.ylabel('cosine-scatter-angle')
plt.xlabel(r'$\Delta$E (meV)')
plt.show()
plt.figure()
plt.hist(mu,bins=80)
plt.semilogy()
plt.xlabel('cosine-scatter-angle')
plt.show()

## Simple usage of Info objects
The `Info` object contain a range of supplementary information about the loaded material. Most notably it includes information about the material density and composition, but other information such as for instance Bragg diffraction structure factors ("HKL lists") or phonon DOS curves, can be access as well. See `help(NC.Info)` for complete details, here we will show a few examples:

In [ ]:
info = NC.createInfo('Al2O3_sg167_Corundum.ncmat')

In [ ]:
info.dump()

In [ ]:
help(info)

In [ ]:
info.composition

In [ ]:
for di in info.dyninfos:
    print(di)

In [ ]:
info.findDynInfo('Al').plot_vdos(unit='THz')#default unit is eV

Or, perhaps you want to see the S(alpha,beta) (a.k.a. S(q,omega)) scattering kernel that this phonon DOS expands to?:

In [ ]:
di.plot_knl(clim=(0,0.2),xlim=(0,500),ylim=(-40,20),phasespace_curves=[1.0])#with 1.0eV neutron kinematic limit shown
di.plot_knl(clim=(0,0.2),xlim=(0,50),ylim=(-4,2),phasespace_curves=[NC.wl2ekin(1.0)])#with 1.0Å neutron kinematic limit shown

In [ ]:
for ai in info.atominfos:
    print(ai)

In [ ]:
import pprint
pprint.pprint(info.findAtomInfo('Al').positions)

In [ ]:
print(info.braggthreshold,'Aa')
print(info.density,'g/cm3')
print(info.numberdensity,'atoms/Aa^3')
print(info.sld,'10^-6/Aa^2')

For the list of HKL reflection planes, one can access them via the `.hklList()` or `.hklObjects()` methods. Note that both of them yield the planes with the longer d-spacings first, so you can always break out of the loop if you wish to forego the planes with shorter d-spacing:

In [ ]:
for hkl in info.hklObjects():
    if hkl.d < 2.2:
        break
    print('-----------------------')
    #print the label, dyspacing, structure factor and multiplicity:
    print(hkl)
    #Access values as needed:
    print(f'   d = {hkl.d:g} Aa')
    print(f'   F^2 = {hkl.f2:g} barn')
    print(f'   multiplicity = {hkl.mult}')
    for h,l,k in zip(hkl.h,hkl.k,hkl.l):
        print(f'   hkl = {h}, {k}, {l}')

As usual we can use the builtin documentation for these objects (using the `next(..)` function to get help for an hkl entry and not for the generator):

In [ ]:
help(next(info.hklObjects()))

## More about cfg-strings
As you have seen, a single NCrystal *cfg-string* like `"somefile.ncmat;par1=val1;par2=val2"` is all that is required to define a material in NCrystal. This is perhaps not very object-oriented or pythonic, but has the advantage that the same string can be used to define a material in a multitude of contexts:

* In Python
* On the command-line
* On a webpage, in an email to a colleage, etc.
* In OpenMC
* In McStas
* In McStasScript
* In Geant4
* ...

Best of all (especially for the NCrystal developers), we can add a new cfg-string parameter without having to update all NCrystal-bindings for OpenMC/McStas/Geant4/... every time. As soon as we add the parameter in NCrystal, all users who have access to the latest NCrystal release, will automatically be able to use the new parameter.

All the parameters are described in the wiki at https://github.com/mctools/ncrystal/wiki/CfgRefDoc but the most important ones are summarised here:

### Most important cfg-parameters

* **Temperature**:
  - Examples: `temp=100`, `temp=100K`, `temp=-10C`, `temp=100F`
  - All materials have a temperature, and it is not always 293.15! The `temp` parameter does exactly what you think it does. By default it assumes the value is in kelvin, but a unit can be added (must be one of `K`, `C`, or `F`.
* **Density**
  - Examples: `density=2.0gcm3`, `density=3.4kgm3`, `density=0.9x`.
  - Also does what you think it does. The last example scales the density by a factor of 0.9.
* **Scattering component toggling**
  - Current recognised component names are `coh_elas` (alias `bragg`), `incoh_elas`, `sans`, and `inelas`. `elas` refers to all components except `inelas`.
  - Syntax: `<compname>=0' (disable component), `comp=<compname1>,..,<compnamen>` (disable all but the listed components.
  - Examples: `...;comp=inelas,sans` (only inelastic and SANS),`...;inelas=0` (without inelastic),`...;inelas=0;comp=inelas` (oups, removed all components)
* **Modify atomic compositions**
  - Examples: `atomdb=H is D`, `atomdb=Al:is:0.9:Al:0.1:Cr`, `atomdb=Si29:28.97649466525u:4.7fm:0.001b:0.101b`, ...
* **Single crystal parameters**:
  - Single crystal models and orientations are primarily controlled by the parameters `mos`, `dir1`, `dir2`, and `dirtol`. Refer to the documentation linked above, and see the example in the next notebook.


For reference, it should be noted that you can also access information about the cfg-string parameters via built-in documentation:
```python
print(NC.generateCfgStrDoc()
```
Or from the command-line (supply `--doc --doc` twice to get full information):
```bash
$> nctool --doc --doc
```

## The nctool (a.k.a. ncrystal_inspectfile) commandline utility
If you are in working in a terminal, the `nctool` or `ncrystal_inspectfile` command (they are the exact same thing, but `nctool` is the future-proof name) provides a convenient interface to a lot of the most common tasks an NCrystal user might need, when composing a cfg-string for their simulation:

* Load the material and do cross section (or scattering) plots or information dumps to the terminal.
  * Investigate the effect of different cfg-parameters.
* Browse available files.
* Extract content of files (including virtual files)
* Quickly unit test an installation

The `nctool` command, **like all NCrystal cmdline tools** contain embedded documentation, accessible by the `--help` flag:

In [ ]:
!nctool --help

In [ ]:
!nctool --dump "Al_sg225.ncmat;temp=200K"

In [ ]:
#Uncomment if you are OK with launching new windows with plots (and it might not work depending on how you run the notebook, but you can always try to run the command in a terminal window)
#!nctool "Al_sg225.ncmat;temp=200K"

In [ ]:
#Uncomment if you are OK with launching new windows with plots (and it might not work depending on how you run the notebook, but you can always try to run the command in a terminal window)
#!nctool "Be_sg194.ncmat;temp=80K" "Be_sg194.ncmat;temp=150K" "Be_sg194.ncmat;temp=300K" --energy

In [ ]:
!nctool --browse

In [ ]:
!nctool --extract Al_sg225.ncmat

## Multiphase materials


Multiphase materials can be composed in cfg-strings from existing materials the following `"phases<...>"` cfg-string syntax, listing the desired phases and their **volume** (not mass) fraction. This is as easy as:
```
   "phases<0.1*PbS_sg225_LeadSulfide.ncmat&0.9*Epoxy_Araldite506_C18H20O3.ncmat>"
```
Cfg-parameters can be applied as always, affecting either a single phase or all phases depending on placement. For instance, in the following (somewhat silly) material the temperature of 300K applies to all phases, while the d-spacing cutoff only affects the aluminium:
```
   "phases<0.25*Al_sg225.ncmat;dcutoff=0.4&0.75*Be_sg194.ncmat>;temp=300K"
```
Note that whitespace is allowed, if you feel it provides a more readable string Also note that the `"phases<>"` syntax has been chosen specifically to support the use-case that one can always override parameters in cfg-strings by appending a simple string to them. For instance appending the string `";temp=400K"` to any cfg-string will override the temperature value - whether or not the cfg-string uses the `"phases<...>"` syntax or not.

For convenience it is also possible to define multiple phases directly inside NCMAT data. Additionally, NCrystal also has framework-level support for SANS physics, which can be thought of as diffraction due to the geometric layouts of the individual phases. We shall see an example in another notebook.

## Single crystal materials
NCrystal supports both isotropic and anisotropic materials. Currently, the only anisotropic models shipped with NCrystal are mosaic single crystals. One is a more generally useful model of Gaussian mosaicity, and the other one is a dedicated one needed to support the special rotated distribution found in pyrolytic graphite (PG). The PG model is automatically enabled when the relevant data file (`C_sg194_pyrolytic_graphite.ncmat`) is used as a single crystal (if not used as a single crystal, it can be used as standard graphite).

Technically, single crystal models are enabled by supplying the material with a mosaic spread (FWHM of the Gaussian mosaicity), and an orientation. The orientation must be specified by providing the coordinates of two vectors in both the crystal space (i.e. the unit cell coordinate system), and the "lab" space (i.e. where the neutron direction is defined). Specifically, we always need the `dir1`, `dir2`, and `mos` to be set, and there are a few other parameters that one might use to fine-tune the models. As always, see the documentation at https://github.com/mctools/ncrystal/wiki/CfgRefDoc .

NCrystal does not provide a simple utility to quickly plot cross sections for anisotropic materials, since the cross sections in general depend on both neutron energy and direction. However, for the case of single crystals used as monochromators, one will often generate *rocking curves*, showing a neutron cross section as a function of incidence angle. If you execute the next code cell below, you will be able to study various monochromator profiles (and then click to the left of the code cell to collapse it and hide the actual code, unless you have a specific interest).

As we can see, the assumption that a monochromator can always be described by a simplistic gaussian is not always very good - especially for neutrons at shorter wavelengths or in monochromators with higher mosaic spreads. Although it should in fairness be noted, that a real monochromator often is accompanied by some sort of angular selection (e.g. a collimator), which makes the plots below appear worse than they are. Still, single crystals can indeed be said to provide rather complicated and interesting cross sections!!

In [ ]:
#
# The code in this cell is very long. You are only meant to study it if you
# have a very specific interest in using single crystals, otherwise you should
# simply click on the left to collapse the cell, and instead play with the resulting
# interactive widget!
#
def get_rocking_curve_of_monochromator( *, dataname, hkl, mosaicity_degree, neutron_wl,
                                        only_selected_plane, absorption ):
    """Extract the rocking curve as (delta_angle,xsect), of the given monochromator. We
    assume the give HKL plane has it's normal orthogonal to the monochromator surface, and
    call that the Z axis. If only_selected_plane=True, the curve will have no contributions
    from other planes or non-Bragg physics."""

    import numpy as np
    #Due to the directional parameters, cfg-strings can get a bit longer than usual when
    #specifying single crystal orientations. Here we first put the material and specify
    #the mosaic spread:
    
    cfgstr = f'{dataname};mos={mosaicity_degree}deg'
    #Let us first put the chosen normal along the Z-axis in the lab:
    cfgstr += ';dir1=@crys_hkl:%i,%i,%i@lab:0,0,1'%hkl
    
    #To completely fix the orientation, we need to provide a second direction as well.
    #For this example we don't care, so we will simply provide a dummy direction. If
    #we specify dirtol=180degrees, then we are effectively telling NCrystal to not
    #complain when it discovers that the angle between dir1 and dir2 is not the same
    #in the crystal and lab systems. Instead NCrystal will instead modify dir2 to a
    #consistent value. Only requirement for this to work is that the dir2 and dir1
    #directions are not parallel. In this simple example function we accomplish
    #that by putting in some silly values:
    cfgstr += ';dir2=@crys_hkl:0.134,0.456,0.6789@lab:0.134,0.456,0.6789'
    cfgstr += ';dirtol=180deg'

    #To figure out the ideal wavelength, we need to know the d-spacing of the chosen
    #reflection plane. Naturally, NCrystal is ready to help:
    d_hkl = NC.createInfo(cfgstr).dspacingFromHKL(*hkl)

    #print(f"Focusing on hkl={hkl} in {filename} with d_hkl={d_hkl:.14g} and neutron wl={neutron_wl}Aa")
    if not neutron_wl < 2*d_hkl:
        print(f'Bragg diffraction on selected plane not possible since neutron_wl>2*d_hkl={2*d_hkl}Å')
        return
    
    if only_selected_plane:
        #Disable other physics, and remove planes not
        #in a narrow range around the selected hkl:
        cfgstr += ';comp=bragg'
        cfgstr += f';dcutoff={(1-1e-9)*d_hkl}'
        cfgstr += f';dcutoffup={(1+1e-9)*d_hkl}'
    
    #print(''.join(cfgstr.splitlines()))
    mat = NC.load(cfgstr)
    selectedhkl = []
    if only_selected_plane:
        for e in mat.info.hklList(all_indices=True):
            selectedhkl += [(h,k,l) for h,k,l in zip(e[0],e[1],e[2])]

    import numpy as np
    mosaicity = mosaicity_degree*np.pi/180.;
    thetabragg = np.arcsin( neutron_wl/(2*d_hkl) )
    def xs_of_rocking_angle(rocking_angle):
        th = thetabragg + rocking_angle
        return mat.scatter.xsect( direction=(np.cos(th),0.0,np.sin(th)), wl=neutron_wl)
    rocking_angles = np.linspace(-mosaicity*10,mosaicity*10,1000)
    xs = np.vectorize(xs_of_rocking_angle)(rocking_angles)
    if absorption:
        xs += mat.absorption.xsect(wl=neutron_wl)
    return dict(curve= ((rocking_angles+thetabragg)*180/np.pi,xs),
                hkl=selectedhkl,
                cfgstr=cfgstr,
                dhkl=d_hkl)
    
materials =[('Ge_sg227.ncmat',((1,1,1),(1,1,3),(1,1,5),(3,3,5))),
            ('C_sg194_pyrolytic_graphite.ncmat',((0,0,2),(0,0,4),(0,0,6),(0,0,8))),
            ('Si_sg227.ncmat',((1,1,1),)),
            ('Cu_sg225.ncmat',((1,1,1),(2,2,2),))]

choices=[]
for name,hkls in materials:
    for hkl in hkls:
        choices.append('hkl=%i%i%i %s'%(*hkl,name))
        
from ipywidgets import interact
@interact(mode=choices,
          mosaicity=['0.001 degree','0.1 degree','0.5 degree'],
          neutron_wavelength=['0.8Å','1.8Å','6Å'])
def plot_rocking_curve(mode='',mosaicity='0.1 degree',neutron_wavelength='1.8Å'):
    import matplotlib.pyplot as plt
    plt.figure('sc_rocking_curve')
    plt.clf()
    neutron_wl=float(neutron_wavelength[:-1])
    dataname=mode.split()[1]
    h,k,l=(int(e) for e in mode.split()[0].split('=')[1])
    curve = lambda osp,absn : get_rocking_curve_of_monochromator( dataname=dataname,
                                                             hkl = (h,k,l),
                                                             mosaicity_degree=float(mosaicity.split()[0]),
                                                             neutron_wl=neutron_wl,
                                                             only_selected_plane = osp,
                                                             absorption = absn)
    
    
    c = curve(True,False)
    if not c:
        return
    plt.plot(*c['curve'],label=f'{len(c["hkl"])*2} selected planes with selected d-spacig')
    c_std = curve(False,False)
    plt.plot(*c_std['curve'],label='+Other scattering')
    plt.plot(*curve(False,True)['curve'],label='+Absorption',ls='--',lw=2)
    plt.grid()
    plt.xlabel('Rocking angle (degree)')
    plt.ylabel('cross section (barn)')
    plt.title(mode.replace('.ncmat',''))
    plt.legend()
    plt.show()
    sp = ') ('.join('%i,%i,%i'%p for p in c['hkl'])
    import textwrap
    l=textwrap.wrap('(%s)'%sp,50)
    print('-'*60)
    print('Cfg-string: "%s"'%c_std['cfgstr'])
    print('d-spacing: "%g Å"'%c_std['dhkl'])
    print('Selected planes: %s'%l[0])
    for e in l[1:]:
        print('                 %s'%e)

## References and where to find more information

The available documentation for NCrystal is a work in progress, and while we still didn't consolidate this in one glorious location, there is plenty of information to be found:

* The official wiki: https://github.com/mctools/ncrystal/wiki/
  * This includes the [Data-library](https://github.com/mctools/ncrystal/wiki/Data-library) and [CfgRefDoc](https://github.com/mctools/ncrystal/wiki/CfgRecDoc) pages which most users might need.
  * For advanced users, the offical [NCMAT format specification](https://github.com/mctools/ncrystal/wiki/NCMAT-format) is there as well, along with instructions for usage or development of optional [plugins](https://github.com/mctools/ncrystal/wiki/Plugins).
  * Be sure to check out the various release Announcement pages you find in the sidebar, since a lot of information was described there but still didn't make it anywhere else.
* The Python API contains doc-strings everywhere, accessible though the `help` function. Please use it!
* All command-line tools support a `--help` flag, which will result in usage instructions being printed out.
* For C++ developers: Lots and lots of thorough comments are left everywhere in the code.
* The [README](https://github.com/mctools/ncrystal/blob/master/README) and [INSTALL](https://github.com/mctools/ncrystal/blob/master/INSTALL) files from the NCrystal release itself.
* Publications (also, **please remember to cite if you use NCrystal and want to support us!**):
  * X.-X. Cai and T. Kittelmann, NCrystal: A library for thermal neutron transport, Computer Physics Communications 246 (2020) 106851, https://doi.org/10.1016/j.cpc.2019.07.015
  * T. Kittelmann and X.-X. Cai, Elastic neutron scattering models for NCrystal, Computer Physics Communications 267 (2021) 108082, https://doi.org/10.1016/j.cpc.2021.108082
  * X.-X. Cai, T. Kittelmann, et. al., "Rejection-based sampling of inelastic neutron scattering", Journal of Computational Physics 380 (2019) 400-407, https://doi.org/10.1016/j.jcp.2018.11.043

## How to get help

There are many options for how to get help with NCrystal. We welcome all contact, but given that we are normally stretched for manpower, some means of contact are certainly easier for us to deal with than others!

* **Using the discussion forum for questions or discussions**
  * Our discussions forum at https://github.com/mctools/ncrystal/discussions is where we hope that users of all levels will ask their questions!
  * Do not be afraid to ask "stupid questions". Such questions are rarely actually stupid, but often expose the need for better documentation, or real bugs in our code. Or perhaps all there is needed is another NCrystal user to share the solution they had in a similar situation.
  * Should a question end up exposing a bug (or feature request) for NCrystal, we can also easily convert them to proper GitHub "issues", and include other relevant experts in the discussions.
  * Feel free to use the forum for other NCrystal-related subjects as well.
  * *Please sign up to "watch" the discussions if you are an NCrystal user that want to contribute by helping other users*
* **Reporting bugs or feature requests**
  * Please do so using the tracker at https://github.com/mctools/ncrystal/issues
  * Don't worry if you don't get the meta-data of the request correct, we can fix it up later. Also don't worry if your issue is a valid one or not - we can always simply provide advice and then close the issue.
  * Do try to include as many details as possible about your issue, so we don't have to drag them out of you.
* **Sending emails to NCrystal developers**
  * Of course you can do so, but please consider first if you could not use one of the other manner of contacts above.
  * Emails can be very distracting, since in practice they must be dealt with either "immediately" or "never".
  * We will never be upset at getting emails, but you might get a short reply asking you to post your problem in one of the GitHub channels instead.
